In [1]:
import numpy as np

from pydrake.all import (DiagramBuilder,
                            LeafSystem, 
                            Context, Parser, MultibodyPlant,
                            AddMultibodyPlantSceneGraph, 
                            StartMeshcat, MeshcatVisualizer, 
                            ExternallyAppliedSpatialForce, SpatialForce
)


In [2]:
meshcat = StartMeshcat()

INFO:drake:Meshcat listening for connections at http://localhost:7001


In [3]:
meshcat.Delete()
builder = DiagramBuilder()


plant, scene_graph = AddMultibodyPlantSceneGraph(builder, time_step=1e-4)
Parser(plant, scene_graph).AddModels('../sdf_model/PRR_planar.sdf')

#Parser(plant, scene_graph).AddModels('../sdf_model/RR_simpler.sdf')
plant.Finalize()


visualizer = MeshcatVisualizer.AddToBuilder(
    builder, 
    scene_graph.get_query_output_port(),
    meshcat
)

diagram = builder.Build()
root_context = diagram.CreateDefaultContext()
plant_context = plant.GetMyContextFromRoot(root_context)

diagram.ForcedPublish(root_context)


In [4]:
plant_context.get_discrete_state_vector().size()


6

In [5]:
plant.CalcMassMatrix(plant_context).reshape(-1)

array([ 3.        ,  0.        ,  0.        ,  0.        ,  2.62499999,
       -1.18749998,  0.        , -1.18749998,  1.0625    ])

In [6]:
plant_context.SetDiscreteState(np.array([0.7, -1.5, 2
                                        ,0, 0, 0]))
diagram.ForcedPublish(root_context)

In [7]:
l3_pose = plant.GetBodyByName('l3')
plant.GetBodyByName('l3').EvalPoseInWorld(plant_context)

RigidTransform(
  R=RotationMatrix([
    [0.00010383364673983404, -0.35078322888327274, -0.9364566810871707],
    [-0.00022596081112234367, -0.9364566704585957, 0.350783199847588],
    [-0.9999999690801421, 0.00017517941238415197, -0.00017649897403413804],
  ]),
  p=[1.0012544248306579, -0.41105169102622824, 0.00011963080756231985],
)

In [16]:
force_obj = ExternallyAppliedSpatialForce()
F = SpatialForce(tau=np.zeros(3), f=np.array([1, 1, 0]))
force_obj.body_index = l3_pose.index()
force_obj.F_Bq_W = F
force_obj.p_BoBq_B = np.array([0, 0, 0.25])

